In [ ]:
import os

from bids import BIDSLayout

In [ ]:
#data_dir = os.path.abspath("../data")
data_dir = "/scratch/mjoseph/rtmsad/bids"
layout = BIDSLayout(data_dir)

In [ ]:
dwi_files = layout.get(datatype="dwi", extension=["nii.gz", "nii"], return_type="file")

In [ ]:
dwi_file = dwi_files[-5]
bvec_file = layout.get_bvec(dwi_file)
bval_file = layout.get_bval(dwi_file)
dwi_meta = layout.get_metadata(dwi_file)

In [ ]:
dwi_file = dwi_files[-5]
bvec_file = layout.get_bvec(dwi_file)
bval_file = layout.get_bval(dwi_file)
dwi_meta = layout.get_metadata(dwi_file)

test_wf = init_dwi_preproc_wf()
test_wf.base_dir = os.getcwd()

inputspec = test_wf.get_node("inputnode")
inputspec.inputs.dwi_file = dwi_file
inputspec.inputs.dwi_meta = dwi_meta
inputspec.inputs.bvec_file = bvec_file
inputspec.inputs.bval_file = bval_file

#test_wf.write_graph(graph2use="flat")
test_wf.write_graph()
test_wf.config["execution"]["remove_unnecessary_outputs"] = False
test_wf.config["execution"]["keep_inputs"] = True
test_wf.config["execution"]["crashfile_format"] = "txt"

test_wf.run()

In [ ]:
for dwi_file in dwi_files:
    bvec_file = layout.get_bvec(dwi_file)
    bval_file = layout.get_bval(dwi_file)
    dwi_meta = layout.get_metadata(dwi_file)
    
    test_wf = init_dwi_preproc_wf()
    test_wf.base_dir = os.getcwd()

    inputspec = test_wf.get_node("inputnode")
    inputspec.inputs.dwi_file = dwi_file
    inputspec.inputs.dwi_meta = dwi_meta
    inputspec.inputs.bvec_file = bvec_file
    inputspec.inputs.bval_file = bval_file

    test_wf.write_graph(graph2use="colored")
    test_wf.config["execution"]["remove_unnecessary_outputs"] = False
    test_wf.config["execution"]["keep_inputs"] = True
    test_wf.config["execution"]["crashfile_format"] = "txt"

    test_wf.run()

In [ ]:
def init_denoise_wf():
    from nipype.pipeline import engine as pe
    from nipype.interfaces import mrtrix3, utility as niu

    wf = pe.Workflow(name="denoise_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file"]), name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    denoise = pe.Node(mrtrix3.DWIDenoise(), name="denoise")
    
    wf.connect([
        (inputnode, denoise, [("dwi_file", "in_file")]),
        (denoise, outputnode, [("out_file", "out_file")])
    ])
    
    return wf

In [ ]:
def init_unring_wf():
    from nipype.pipeline import engine as pe
    from nipype.interfaces import mrtrix3, utility as niu

    wf = pe.Workflow(name="unring_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file"]), name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    unring = pe.Node(mrtrix3.MRDeGibbs(), name="unring")
    
    wf.connect([
        (inputnode, unring, [("dwi_file", "in_file")]),
        (unring, outputnode, [("out_file", "out_file")])
    ])
    
    return wf

In [ ]:
def init_eddy_wf():
    from nipype.pipeline import engine as pe
    from nipype.interfaces import fsl, utility as niu
    
    wf = pe.Workflow(name="eddy_wf")

    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file",
                                                      "bvec_file",
                                                      "bval_file",
                                                      "mask_file",
                                                      "acqp",
                                                      "index"]),
                       name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file", "out_bvec"]), name="outputnode")
    
    ecc = pe.Node(
        fsl.Eddy(num_threads=1, repol=True, cnr_maps=True, residuals=True),
        name="fsl_eddy",
    )
    
    #eddy_quad = pe.Node(fsl.EddyQuad(verbose=True), name="eddy_quad")
    
    wf.connect([
        (inputnode, ecc, [("dwi_file", "in_file"),
                           ("bval_file", "in_bval"),
                           ("bvec_file", "in_bvec"),
                           ("acqp", "in_acqp"),
                           ("index", "in_index"),
                           ("mask_file", "in_mask")]),
        (ecc, outputnode, [("out_corrected", "out_file"),
                           ("out_rotated_bvecs", "out_bvec")])
        ])
    
    return wf

In [ ]:
def init_tensor_wf():
    from nipype.pipeline import engine as pe
    from nipype.interfaces import fsl, utility as niu
    
    wf = pe.Workflow(name="tensor_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", "bvec_file", "bval_file", "mask_file"]), 
                        name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["FA_file", "V1_file"]), name="outputnode")
    
    dtifit = pe.Node(fsl.DTIFit(save_tensor=True, sse=True), name="dtifit")
    
    wf.connect([
        (inputnode, dtifit, [("dwi_file", "dwi"),
                             ("bvec_file", "bvecs"),
                             ("bval_file", "bvals"),
                             ("mask_file", "mask")]),
        (dtifit, outputnode, [("FA", "FA_file"),
                              ("V1", "V1_file")])
    ])
    
    return wf

In [ ]:
def init_dwi_preproc_wf():
    from nipype.pipeline import engine as pe
    from nipype.interfaces import ants, fsl, utility as niu
    
    wf = pe.Workflow(name="dwi_preproc_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", 
                                                      "dwi_meta", 
                                                      "bvec_file", 
                                                      "bval_file"]), 
                        name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    def gen_index(in_file):
        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        out_file = fname_presuffix(
            in_file,
            suffix="_index.txt",
            newpath=os.path.abspath("."),
            use_ext=False)
        
        vols = nib.load(in_file, mmap=NUMPY_MMAP).get_data().shape[-1]
        index_lines = np.ones((vols,))
        index_lines_reshape = index_lines.reshape(1, index_lines.shape[0])
        np.savetxt(out_file, index_lines_reshape, fmt="%i")
        return out_file

    gen_idx = pe.Node(
        niu.Function(
            input_names=["in_file"],
            output_names=["out_file"],
            function=gen_index),
        name="gen_index")
    
    def gen_acqparams(in_file, metadata):
        import os
        from nipype.utils.filemanip import fname_presuffix

        out_file = fname_presuffix(
            in_file,
            suffix="_acqparams.txt",
            newpath=os.path.abspath("."),
            use_ext=False)

        acq_param_dict = {
            "j": "0 1 0 %.7f",
            "j-": "0 -1 0 %.7f",
            "i": "1 0 0 %.7f",
            "i-": "-1 0 0 %.7f",
            "k": "0 0 1 %.7f",
            "k-": "0 0 -1 %.7f"}
        
        pe_dir = metadata.get("PhaseEncodingDirection")
        total_readout = metadata.get("TotalReadoutTime")
        acq_params = acq_param_dict[pe_dir] % total_readout
        
        with open(out_file, "w") as f:
            f.write(acq_params)

        return out_file

    acqp = pe.Node(
        niu.Function(
            input_names=["in_file", "metadata"],
            output_names=["out_file"],
            function=gen_acqparams),
        name="acqp")
    
    wf.connect([
        (inputnode, gen_idx, [("dwi_file", "in_file")]),
        (inputnode, acqp, [("dwi_file", "in_file"),
                           ("dwi_meta", "metadata")])
    ])
    
    denoise_wf = init_denoise_wf()
    unring_wf = init_unring_wf()
    
    wf.connect([
        (inputnode, denoise_wf, [("dwi_file", "inputnode.dwi_file")]),
        (denoise_wf, unring_wf, [("outputnode.out_file", "inputnode.dwi_file")])
    ])
    
    def b0_average(in_dwi, in_bval, b0_thresh=0, out_file=None):
        """
        A function that averages the *b0* volumes from a DWI dataset.
        As current dMRI data are being acquired with all b-values > 0.0,
        the *lowb* volumes are selected by specifying the parameter b0_thresh.
        .. warning:: *b0* should be already registered (head motion artifact
        should be corrected).
        """
        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        if out_file is None:
            out_file = fname_presuffix(
                in_dwi, suffix="_avg_b0", newpath=os.path.abspath(".")
            )

        imgs = np.array(nib.four_to_three(nib.load(in_dwi, mmap=NUMPY_MMAP)))
        bval = np.loadtxt(in_bval)
        index = np.argwhere(bval <= b0_thresh).flatten().tolist()

        b0s = [im.get_data().astype(np.float32) for im in imgs[index]]
        b0 = np.average(np.array(b0s), axis=0)

        hdr = imgs[0].header.copy()
        hdr.set_data_shape(b0.shape)
        hdr.set_xyzt_units("mm")
        hdr.set_data_dtype(np.float32)
        nib.Nifti1Image(b0, imgs[0].affine, hdr).to_filename(out_file)
        return out_file

    avg_b0_0 = pe.Node(
        niu.Function(
            input_names=["in_dwi", "in_bval", "b0_thresh"],
            output_names=["out_file"],
            function=b0_average),
        name="b0_avg_pre")

    bet_dwi0 = pe.Node(
        fsl.BET(frac=0.6, mask=True, robust=True),
        name="bet_dwi_pre"
    )
    
    wf.connect([
        (unring_wf, avg_b0_0, [("outputnode.out_file", "in_dwi")]),
        (inputnode, avg_b0_0, [("bval_file", "in_bval")]),
        (avg_b0_0, bet_dwi0, [("out_file", "in_file")])
    ])
    
    eddy_wf = init_eddy_wf()
    
    wf.connect([
        (unring_wf, eddy_wf, [("outputnode.out_file", "inputnode.dwi_file")]),
        (inputnode, eddy_wf, [("bval_file", "inputnode.bval_file"),
                              ("bvec_file", "inputnode.bvec_file")]),
        (bet_dwi0, eddy_wf, [("mask_file", "inputnode.mask_file")]),
        (gen_idx, eddy_wf, [("out_file", "inputnode.index")]),
        (acqp, eddy_wf, [("out_file", "inputnode.acqp")])
    ])
 
    fslroi = pe.Node(
        fsl.ExtractROI(t_min=0, t_size=1),
        name="fslroi"
    )
    
    bias_correct = pe.Node(
        ants.N4BiasFieldCorrection(save_bias=True, copy_header=True, dimension=3),
        name="bias_correct"
    )

    def get_b0_mask_fn(b0_file):
        import os
        import nibabel as nib
        from nipype.utils.filemanip import fname_presuffix
        from dipy.segment.mask import median_otsu

        mask_file = fname_presuffix(
            b0_file, suffix="_mask", newpath=os.path.abspath(".")
        )
        img = nib.load(b0_file)
        data, aff = img.get_data(), img.affine
        _, mask = median_otsu(data, 2, 1)
        nib.Nifti1Image(mask.astype(float), aff).to_filename(mask_file)
        return mask_file

    b0mask_node = pe.Node(
        niu.Function(
            input_names=["b0_file"],
            output_names=["mask_file"],
            function=get_b0_mask_fn),
        name="getB0Mask")
    
    wf.connect([
        (eddy_wf, fslroi, [("outputnode.out_file", "in_file")]),
        (fslroi, bias_correct, [("roi_file", "input_image")]),
        (bias_correct, b0mask_node, [("output_image", "b0_file")])
    ])
    
    tensor_wf = init_tensor_wf()
    
    wf.connect([
        (inputnode, tensor_wf, [("bval_file", "inputnode.bval_file")]),
        (b0mask_node, tensor_wf, [("mask_file", "inputnode.mask_file")]),
        (eddy_wf, tensor_wf, [("outputnode.out_file", "inputnode.dwi_file"),
                              ("outputnode.out_bvec", "inputnode.bvec_file")])
    ])

    return wf

In [ ]:
def init_eddy_wf():
    
    wf = pe.Workflow(name="eddy_wf")

    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file",
                                                      "bvec_file",
                                                      "bval_file",
                                                      "mask_file",
                                                      "acqp",
                                                      "index",
                                                      "fieldcoef",
                                                      "movpar"]))
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    ecc = pe.Node(
        fsl.Eddy(repol=True, cnr_maps=True, residuals=True),
        name="fsl_eddy",
    )

    # if num_threads not specified, do this
    import multiprocessing

    ecc.inputs.num_threads = multiprocessing.cpu_count()

    try:
        if cuda.gpus:
            ecc.inputs.use_cuda = True
    except:
        ecc.inputs.use_cuda = False
    
    wf.connect(
        [(inputnode, ecc, [("dwi_file", "in_file"),
                           ("bval_file", "in_bval"),
                           ("bvec_file", "in_bvec"),
                           ("acqp", "in_acqp"),
                           ("index", "in_index"),
                           ("mask_file", "in_mask"),
                           ("fieldcoef", "in_topup_fieldcoef"),
                           ("movpar", "in_topup_movpar")])
        ]
    )
    
    return wf

In [ ]:
def init_artefact_removal_wf(ignore):
    from nipype.pipeline import engine as pe
    from nipype.interfaces import mrtrix3, utility as niu

    wf = pe.Workflow(name="artefact_remove_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file"]))
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    denoise = pe.Node(mrtrix3.DWIDenoise(), name="denoise")
    
    unring = pe.Node(mrtrix3.MRDeGibbs(), name="unring")
    
    ignore_list = list(ignore)
    
    if ignore_list == ["denoise"]:
        wf.connect([
            (inputnode, unring, [("dwi_file", "in_file")]),
            (unring, outputnode, [("out_file", "out_file")])
        ])
    
    elif ignore_list == ["unring"]:
        wf.connect([
            (inputnode, denoise, [("dwi_file", "in_file")]),
            (denoise, outputnode, [("out_file", "out_file")])
        ])

    elif ["denoise", "unring"] in ignore_list:
        wf.connect([
            (inputnode, outputnode, "dwi_file", "out_file")
        ])
    
    else:
        wf.connect([
            (inputnode, denoise, [("dwi_file", "in_file")]),
            (denoise, unring, [("out_file", "in_file")]),
            (unring, outputnode, [("out_file", "out_file")])
        ])
    
    return wf

In [ ]:
def init_sdc_wf(layout):
    wf = pe.Workflow(name="sdc_wf")

    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", "dwi_meta", "fmap_files"]))
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    FMAP_PRIORITY = {"epi": 0, "fieldmap": 1, "phasediff": 2, "phase": 3, "syn": 4}
    
    fmaps.sort(key=lambda fmap: FMAP_PRIORITY[fmap["suffix"]])
    fmap = fmaps[0]
    
    if fmap["suffix"] in ("fieldmap", "phasediff", "phase"):
        fmap_type = "field"
        
        if fmap["suffix"] == "fieldmap":
            
            

    elif fmap["suffix"] == "epi":
        fmap_type = "topup"
        
        epi_fmaps = [fmap_["epi"] for fmap_ in fmaps if fmap_["suffix"] == "epi"]
           
    def gen_acqparams(in_files):
        import os
        from nipype.utils.filemanip import fname_presuffix

        out_file = fname_presuffix(
            in_file,
            suffix="_acqparams.txt",
            newpath=os.path.abspath("."),
            use_ext=False)

        acq_param_dict = {
            "j": "0 1 0 %.7f",
            "j-": "0 -1 0 %.7f",
            "i": "1 0 0 %.7f",
            "i-": "-1 0 0 %.7f",
            "k": "0 0 1 %.7f",
            "k-": "0 0 -1 %.7f"}
        
        acq_param_lines = []
        
        for f in in_files:
            metadata = layout.get_metadata(f)
            pe_dir = metadata.get("PhaseEncodingDirection")
            total_readout = metadata.get("TotalReadoutTime")
            acq_param_lines.append(acq_param_dict[pe_dir] % total_readout)
        
        acq_params = '\n'.join((acq_param_lines))

        with open(out_file, "w") as f:
            f.write(acq_params)

        return out_file

    acqp = pe.Node(
        niu.Function(
            input_names=["in_file", "metadata"],
            output_names=["out_file"],
            function=gen_acqparams,
        ),
        name="acqp",
    )
    
    return wf

In [ ]:
def init_phasediff_wf(layout):
    wf = pe.Workflow(name="phasediff_wf")
    
    

In [ ]:
def init_pepolar_wf():
    wf = pe.Workflow(name="pepolar_wf")

    list_merge = pe.Node(niu.Merge(numinputs=2), name="list_merge")

    merge = pe.Node(fsl.Merge(dimension="t"), name="mergeAPPA")

    topup = pe.Node(fsl.TOPUP(), name="topup")
    
    wf.connect(
        [(inputnode, acqp, [("dwi_file", "in_file"),
                            ("dwi_meta", "metadata")]),
         (inputnode, list_merge, [("ap_file", "in1"),
                                  ("pa_file", "in2")]),
         (list_merge, merge, [("out", "in_files")]),
         (merge, topup, [("merged_file", "in_file")]),
         (acqp, topup, [("out_file", "encoding_file")]),
         (topup, outputnode, [("", "")])
        ]
    )
    
    return wf

In [ ]:
def init_topup_wf():
    wf = pe.Workflow(name="topup_wf")

    inputnode = pe.Node(niu.IdentityInterface(fields=["ap_file", "pa_file", "acqp"]))
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
       
    list_merge = pe.Node(niu.Merge(numinputs=2), name="list_merge")

    merge = pe.Node(fsl.Merge(dimension="t"), name="mergeAPPA")

    topup = pe.Node(fsl.TOPUP(), name="topup")
    
    wf.connect(
        [(inputnode, list_merge, [("ap_file", "in1"),
                                  ("pa_file", "in2")]),
         (list_merge, merge, [("out", "in_files")]),
         (merge, topup, [("merged_file", "in_file")]),
         (inputnode, topup, [("acqp", "encoding_file")])
        ]
    )
    
    return wf

In [ ]:
def init_fmap_wf():

    wf = pe.Workflow(name="fmap_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["fmap_file", "mag_file", "b0"]))
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    wf.connect(
        [(inputnode, rad_to_hz, [("fmap_file", "in_file")]),
         (inputnode, mag_flirt, [("mag_file", "in_file"),
                                 ("b0", "reference")]),
         (rad_to_hz, fmap_flirt, [("out_file", "in_file")]),
         (inputnode, fmap_flirt, [("b0", "reference")]),
         (mag_flirt, fmap_flirt, [("out_matrix_file", "in_matrix_file")])
        ]
    )
    
    return wf

In [ ]:
def init_eddy_wf():
    
    wf = pe.Workflow(name="eddy_wf")

    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file",
                                                      "bvec_file",
                                                      "bval_file",
                                                      "mask_file",
                                                      "acqp",
                                                      "index",
                                                      "fieldcoef",
                                                      "movpar"]))
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    ecc = pe.Node(
        fsl.Eddy(repol=True, cnr_maps=True, residuals=True),
        name="fsl_eddy",
    )

    # if num_threads not specified, do this
    import multiprocessing

    ecc.inputs.num_threads = multiprocessing.cpu_count()

    try:
        if cuda.gpus:
            ecc.inputs.use_cuda = True
    except:
        ecc.inputs.use_cuda = False
    
    wf.connect(
        [(inputnode, ecc, [("dwi_file", "in_file"),
                           ("bval_file", "in_bval"),
                           ("bvec_file", "in_bvec"),
                           ("acqp", "in_acqp"),
                           ("index", "in_index"),
                           ("mask_file", "in_mask"),
                           ("fieldcoef", "in_topup_fieldcoef"),
                           ("movpar", "in_topup_movpar")])
        ]
    )
    
    return wf

In [ ]:
def init_dwi_preproc_wf(ignore):
    from nipype.pipeline import engine as pe
    from nipype.interfaces import ants, fsl, utility as niu
    
    wf = pe.Workflow(name="dwi_preproc_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", 
                                                      "dwi_meta", 
                                                      "bvec_file", 
                                                      "bval_file", 
                                                      "ap_file", 
                                                      "pa_file"]), 
                        name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    def gen_index(in_file):
        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        out_file = fname_presuffix(
            in_file,
            suffix="_index.txt",
            newpath=os.path.abspath("."),
            use_ext=False)
        
        vols = nib.load(in_file, mmap=NUMPY_MMAP).get_data().shape[-1]
        index_lines = np.ones((vols,))
        index_lines_reshape = index_lines.reshape(1, index_lines.shape[0])
        np.savetxt(out_file, index_lines_reshape, fmt="%i")
        return out_file

    gen_idx = pe.Node(
        niu.Function(
            input_names=["in_file"],
            output_names=["out_file"],
            function=gen_index),
        name="gen_index")
    
    artefact_removal_wf = init_artefact_removal_wf(ignore)

    def b0_average(in_dwi, in_bval, b0_thresh=0, out_file=None):
        """
        A function that averages the *b0* volumes from a DWI dataset.
        As current dMRI data are being acquired with all b-values > 0.0,
        the *lowb* volumes are selected by specifying the parameter b0_thresh.
        .. warning:: *b0* should be already registered (head motion artifact
        should be corrected).
        """
        import os
        import numpy as np
        import nibabel as nib
        from nipype.utils import NUMPY_MMAP
        from nipype.utils.filemanip import fname_presuffix

        if out_file is None:
            out_file = fname_presuffix(
                in_dwi, suffix="_avg_b0", newpath=os.path.abspath(".")
            )

        imgs = np.array(nib.four_to_three(nib.load(in_dwi, mmap=NUMPY_MMAP)))
        bval = np.loadtxt(in_bval)
        index = np.argwhere(bval <= b0_thresh).flatten().tolist()

        b0s = [im.get_data().astype(np.float32) for im in imgs[index]]
        b0 = np.average(np.array(b0s), axis=0)

        hdr = imgs[0].header.copy()
        hdr.set_data_shape(b0.shape)
        hdr.set_xyzt_units("mm")
        hdr.set_data_dtype(np.float32)
        nib.Nifti1Image(b0, imgs[0].affine, hdr).to_filename(out_file)
        return out_file

    avg_b0_0 = pe.Node(
        niu.Function(
            input_names=["in_dwi", "in_bval", "b0_thresh"],
            output_names=["out_file"],
            function=b0_average,
        ),
        name="b0_avg_pre"
    )

    bet_dwi0 = pe.Node(
        fsl.BET(frac=0.6, mask=True, robust=True),
        name="bet_dwi_pre"
    )
    
    sdc_wf = init_sdc_wf()
    
    eddy_wf = init_eddy_wf()
    
    bias_correct = pe.Node(
        ant.N4BiasCorrect(dimension=3),
        name="bias_correct"
    )
    
    wf.connect(
        [(inputnode, artefact_removal_wf, ["dwi_file", "dwi_file"])
         (inputnode, gen_idx, [("dwi_file", "in_file")]),
         (inputnode, avg_b0_0, [("bval_file", "in_bval")]),
         (artefact_removal_wf, avg_b0_0, [("out_file", "in_dwi")]),
         (avg_b0_0, bet_dwi0, [("out_file", "in_file")])
        ]
    )
    
    return wf